In [27]:
import s3fs
import pyarrow.dataset as ds
import pandas as pd
import gc  # Import du module pour gérer la mémoire

# Créer un système de fichiers S3 avec l'URL de MinIO
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

# Lister les fichiers Parquet dans le dossier
files = ["aymericp/CNAM_parquet/A2021"+(("0"+str(m))[-2:])+ ".parquet" for m in range(1,13)]  # Trouve tous les fichiers .parquet dans le dossier

# Initialiser une liste pour stocker les DataFrames
dfs = []

# Pour chaque fichier Parquet
for file in files:
    print(file)
    # Créer un dataset PyArrow à partir du fichier Parquet
    dataset = ds.dataset(f"s3://{file}", filesystem=fs, format="parquet")
    
    # Appliquer un filtre pour ne lire que les lignes où PRS_NAT == 2331 et ASU_NAT == 40
    filtered_table = dataset.to_table(
        columns=['PRS_REM_MNT','AGE_BEN_SNDS','ASU_NAT','PRS_NAT','PSE_ACT_CAT','PSE_ACT_SNDS','PSE_SPE_SNDS'],  # Sélectionner les colonnes nécessaires
        #filter=(ds.field('ASU_NAT') == 40)  # Appliquer le filtre
    )
    
    # Convertir la table PyArrow en DataFrame Pandas
    df_filtered = filtered_table.to_pandas()
    df_filtered = df_filtered.groupby(['AGE_BEN_SNDS','ASU_NAT','PRS_NAT','PSE_ACT_CAT','PSE_ACT_SNDS','PSE_SPE_SNDS']).sum().reset_index()
    
    # Ajouter le DataFrame filtré à la liste
    dfs.append(df_filtered)
    
    # Libérer la mémoire en supprimant l'objet DataFrame et en effectuant un ramassage de la mémoire
    del df_filtered  # Supprimer l'objet DataFrame
    gc.collect()  # Forcer le ramassage de la mémoire

# Concatenation des DataFrames filtrés
final_df = pd.concat(dfs, ignore_index=True)
final_df = final_df.groupby(['AGE_BEN_SNDS','ASU_NAT','PRS_NAT','PSE_ACT_CAT','PSE_ACT_SNDS','PSE_SPE_SNDS']).sum().reset_index()

# Afficher le DataFrame final
print(final_df)

final_df.to_csv('BDconcat21.csv',sep=';')


aymericp/CNAM_parquet/A202101.parquet
aymericp/CNAM_parquet/A202102.parquet
aymericp/CNAM_parquet/A202103.parquet
aymericp/CNAM_parquet/A202104.parquet
aymericp/CNAM_parquet/A202105.parquet
aymericp/CNAM_parquet/A202106.parquet
aymericp/CNAM_parquet/A202107.parquet
aymericp/CNAM_parquet/A202108.parquet
aymericp/CNAM_parquet/A202109.parquet
aymericp/CNAM_parquet/A202110.parquet
aymericp/CNAM_parquet/A202111.parquet
aymericp/CNAM_parquet/A202112.parquet
       AGE_BEN_SNDS  ASU_NAT  PRS_NAT  PSE_ACT_CAT  PSE_ACT_SNDS  \
0                 0       10     1089            1             0   
1                 0       10     1090            0             0   
2                 0       10     1090            0             0   
3                 0       10     1090            1             0   
4                 0       10     1090            1             0   
...             ...      ...      ...          ...           ...   
47035            99       99     3211            0             0   


In [2]:
import s3fs
import pyarrow.dataset as ds
import pandas as pd
import gc  # Import du module pour gérer la mémoire

# Créer un système de fichiers S3 avec l'URL de MinIO
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

def extractionN(N):
    # Lister les fichiers Parquet dans le dossier
    files = ["aymericp/CNAM_parquet/A"+str(N)+(("0"+str(m))[-2:])+ ".parquet" for m in range(1,13)]  # Trouve tous les fichiers .parquet dans le dossier
    
    # Initialiser une liste pour stocker les DataFrames
    dfs = []
    # Pour chaque fichier Parquet
    for file in files:
        print(file)
        # Créer un dataset PyArrow à partir du fichier Parquet
        dataset = ds.dataset(f"s3://{file}", filesystem=fs, format="parquet")
        
        # Appliquer un filtre pour ne lire que les lignes où PRS_NAT == 2331 et ASU_NAT == 40
        filtered_table = dataset.to_table(
            columns=['PRS_REM_MNT','AGE_BEN_SNDS','CPT_ENV_TYP','ASU_NAT','PRS_NAT'],  # Sélectionner les colonnes nécessaires
            filter=(ds.field('ASU_NAT') == 40)  # Appliquer le filtre
        )
        
        # Convertir la table PyArrow en DataFrame Pandas
        df_filtered = filtered_table.to_pandas()
        df_filtered['PRS_NAT'] = df_filtered['PRS_NAT'].apply(lambda x: 0 if 0 <= x <= 1999 else 1 if 2000 <= x <= 2999 else 0 if 3000 <= x <= 5999 else 2 if 6000 <= x <= 6999 else 3 if 7000 <= x <= 8999 else 4)
        df_filtered = df_filtered[df_filtered['PRS_NAT']<2]
        df_filtered = df_filtered.groupby(['AGE_BEN_SNDS','CPT_ENV_TYP','ASU_NAT','PRS_NAT']).sum().reset_index()
        
        # Ajouter le DataFrame filtré à la liste
        dfs.append(df_filtered)
        
        # Libérer la mémoire en supprimant l'objet DataFrame et en effectuant un ramassage de la mémoire
        del df_filtered  # Supprimer l'objet DataFrame
        gc.collect()  # Forcer le ramassage de la mémoire
    
    # Concatenation des DataFrames filtrés
    final_df = pd.concat(dfs, ignore_index=True)
    final_df = final_df.groupby(['AGE_BEN_SNDS','ASU_NAT','CPT_ENV_TYP','PRS_NAT']).sum().reset_index()
    return(final_df)

for i in range(2015,2016):
    extractionN(i).to_csv('Concat'+str(i)+'.csv',sep=';',index=False)


aymericp/CNAM_parquet/A201501.parquet
aymericp/CNAM_parquet/A201502.parquet
aymericp/CNAM_parquet/A201503.parquet
aymericp/CNAM_parquet/A201504.parquet
aymericp/CNAM_parquet/A201505.parquet
aymericp/CNAM_parquet/A201506.parquet
aymericp/CNAM_parquet/A201507.parquet
aymericp/CNAM_parquet/A201508.parquet
aymericp/CNAM_parquet/A201509.parquet
aymericp/CNAM_parquet/A201510.parquet
aymericp/CNAM_parquet/A201511.parquet
aymericp/CNAM_parquet/A201512.parquet


In [6]:
import s3fs
import pyarrow.dataset as ds
import pandas as pd
import gc

# Créer un système de fichiers S3 avec l'URL de MinIO
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

def extractionN(N):
    files = [f"aymericp/CNAM_parquet/A{N}{str(m).zfill(2)}.parquet" for m in range(1, 13)]
    dfs = []

    for i, file in enumerate(files):
        if i % 3 == 0:
            print(f"Lecture : {file}")

        # Scanner rapide avec colonnes et filtres
        dataset = ds.dataset(f"s3://{file}", filesystem=fs, format="parquet")
        scanner = dataset.scanner(
            columns=['PRS_REM_MNT', 'AGE_BEN_SNDS', 'CPT_ENV_TYP', 'ASU_NAT', 'PRS_NAT'],
            filter=(ds.field('ASU_NAT') == 40) & (ds.field('PRS_NAT') < 9000)
        )
        table = scanner.to_table()

        # Conversion en DataFrame
        df = table.to_pandas(use_threads=True)

        # Transformation rapide de PRS_NAT
        df['PRS_NAT'] = df['PRS_NAT'].apply(lambda x: 0 if 0 <= x <= 1999 else 1 if 2000 <= x <= 2999 else 0 if 3000 <= x <= 5999 else 2 if 6000 <= x <= 6999 else 3)
        df = df.groupby(['AGE_BEN_SNDS', 'CPT_ENV_TYP', 'ASU_NAT', 'PRS_NAT']).sum().reset_index()
        dfs.append(df)

        del df, table
        gc.collect()

    final_df = pd.concat(dfs, ignore_index=True)
    final_df = final_df.groupby(['AGE_BEN_SNDS', 'ASU_NAT', 'CPT_ENV_TYP', 'PRS_NAT']).sum().reset_index()
    return final_df

# Boucle principale
for i in range(2017, 2025):
    df_annuel = extractionN(i)
    df_annuel.to_csv(f'Concat{i}.csv', sep=';', index=False)


Lecture : aymericp/CNAM_parquet/A201701.parquet
Lecture : aymericp/CNAM_parquet/A201704.parquet
Lecture : aymericp/CNAM_parquet/A201707.parquet
Lecture : aymericp/CNAM_parquet/A201710.parquet
Lecture : aymericp/CNAM_parquet/A201801.parquet
Lecture : aymericp/CNAM_parquet/A201804.parquet
Lecture : aymericp/CNAM_parquet/A201807.parquet
Lecture : aymericp/CNAM_parquet/A201810.parquet
Lecture : aymericp/CNAM_parquet/A201901.parquet
Lecture : aymericp/CNAM_parquet/A201904.parquet
Lecture : aymericp/CNAM_parquet/A201907.parquet
Lecture : aymericp/CNAM_parquet/A201910.parquet
Lecture : aymericp/CNAM_parquet/A202001.parquet
Lecture : aymericp/CNAM_parquet/A202004.parquet
Lecture : aymericp/CNAM_parquet/A202007.parquet
Lecture : aymericp/CNAM_parquet/A202010.parquet
Lecture : aymericp/CNAM_parquet/A202101.parquet
Lecture : aymericp/CNAM_parquet/A202104.parquet
Lecture : aymericp/CNAM_parquet/A202107.parquet
Lecture : aymericp/CNAM_parquet/A202110.parquet
Lecture : aymericp/CNAM_parquet/A202201.

In [3]:
concat1524 = pd.read_csv('Concat2015.csv',sep=';')

for i in range(2016,2025):
    concat1524 = concat1524.merge(pd.read_csv('Concat'+str(i)+'.csv',sep=';'),how='outer',on=['AGE_BEN_SNDS','ASU_NAT','CPT_ENV_TYP','PRS_NAT'],suffixes = ('_'+str(i-1),'_'+str(i)))
    concat1524 = concat1524.fillna(0)

In [4]:
pop = pd.read_csv('pop20102070.csv', sep=';')
pop = pop[['Age', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028']]
pop['Age'] = pop['Age'].replace("105+","105").astype(int)
# Définir les bornes des groupes d'âges
bins = [-1, 19, 29, 39, 49, 59, 69, 79, 89, float('inf')]
# Définir les labels pour chaque groupe d'âge
labels = [0, 20, 30, 40, 50, 60, 70, 80, 99]

# Appliquer pd.cut() pour créer une nouvelle colonne avec les groupes d'âges
pop['AgeGroup'] = pd.cut(pop['Age'], bins=bins, labels=labels, right=True)

# Supprimer la colonne 'Age'
pop = pop.drop(columns=['Age'])

# Regrouper par 'AgeGroup' et sommer les autres colonnes
pop_grouped = pop.groupby('AgeGroup').sum().reset_index()

popetremb = pd.merge(concat1524,pop_grouped,how='outer',left_on = 'AGE_BEN_SNDS', right_on='AgeGroup')

popetremb['ratio2015'] = popetremb['PRS_REM_MNT_2015']/popetremb['2015']
popetremb['ratio2016'] = popetremb['PRS_REM_MNT_2016']/popetremb['2016']
popetremb['ratio2017'] = popetremb['PRS_REM_MNT_2017']/popetremb['2017']
popetremb['ratio2018'] = popetremb['PRS_REM_MNT_2018']/popetremb['2018']
popetremb['ratio2019'] = popetremb['PRS_REM_MNT_2019']/popetremb['2019']
popetremb['ratio2022'] = popetremb['PRS_REM_MNT_2022']/popetremb['2022']
popetremb['ratio2023'] = popetremb['PRS_REM_MNT_2023']/popetremb['2023']
popetremb['ratio2024'] = popetremb['PRS_REM_MNT_2024']/popetremb['2024']
popetremb['ratio2020'] = 2/3*popetremb['ratio2019']+1/3*popetremb['ratio2022']
popetremb['ratio2021'] = 1/3*popetremb['ratio2019']+2/3*popetremb['ratio2022']

print(popetremb)

    AGE_BEN_SNDS  ASU_NAT  CPT_ENV_TYP  PRS_NAT  PRS_REM_MNT_2015  \
0              0       40            0        0         169216.15   
1              0       40            0        1          50277.38   
2              0       40            0        2              0.00   
3              0       40            0        3              0.00   
4              0       40            1        0        6009471.21   
..           ...      ...          ...      ...               ...   
159           99       40            8        1             36.00   
160           99       40            9        0         765888.84   
161           99       40            9        1          42986.58   
162           99       40            9        2              0.00   
163           99       40            9        3              0.00   

     PRS_REM_MNT_2016  PRS_REM_MNT_2017  PRS_REM_MNT_2018  PRS_REM_MNT_2019  \
0           161221.58         148207.40         140027.57         100346.69   
1            

/tmp/ipykernel_1266/2083965888.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pop_grouped = pop.groupby('AgeGroup').sum().reset_index()


In [5]:
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
# Désactiver le warning de downcasting
pd.set_option('future.no_silent_downcasting', True)

# Liste des colonnes de ratios à utiliser pour la modélisation
ratios_columns = ['ratio2015', 'ratio2016', 'ratio2017', 'ratio2018', 'ratio2019', 'ratio2020', 'ratio2021', 'ratio2022', 'ratio2023', 'ratio2024']

# Créer un DataFrame vide pour stocker les prédictions
predictions_df = pd.DataFrame(columns=['ratio2025', 'ratio2026', 'ratio2027', 'ratio2028'])

for col in ratios_columns:
    popetremb[col] = pd.to_numeric(popetremb[col], errors='coerce')

# Appliquer le modèle ARIMA par ligne
for index, row in popetremb.iterrows():
    # Extraire les données de la ligne (les ratios de 2017 à 2024)
    data = row[ratios_columns]
    data = data.fillna(0) 
    data = np.asarray(data, dtype=float)

    # Créer le modèle ARIMA(1, 1, 0)
    model = ARIMA(data, order=(1, 1, 0))
    model_fit = model.fit()
    
    # Prédire les valeurs futures pour 2025, 2026, 2027, 2028
    forecast = model_fit.forecast(steps=4)
    
    # Créer un DataFrame avec les prédictions pour cette ligne
    forecast_df = pd.DataFrame({
        'ratio2025': [forecast[0]],
        'ratio2026': [forecast[1]],
        'ratio2027': [forecast[2]],
        'ratio2028': [forecast[3]]
    })
        
    # Utiliser pd.concat pour ajouter les prédictions au DataFrame final
    predictions_df = pd.concat([predictions_df, forecast_df], ignore_index=True)
    
# Afficher les résultats des prédictions
print(predictions_df)

/tmp/ipykernel_1266/2486996054.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_df = pd.concat([predictions_df, forecast_df], ignore_index=True)
/usr/local/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Ch

        ratio2025     ratio2026     ratio2027     ratio2028
0    3.066482e-03  2.969900e-03  2.910705e-03  2.874424e-03
1    6.252094e-04  6.662724e-04  6.572324e-04  6.592225e-04
2   -7.139473e-23 -6.861985e-23 -6.872770e-23 -6.872351e-23
3    3.227521e-04  3.250121e-04  3.247414e-04  3.247738e-04
4    3.533956e-01  3.534202e-01  3.534223e-01  3.534225e-01
..            ...           ...           ...           ...
159 -1.203034e-88 -1.203034e-88 -1.203034e-88 -1.203034e-88
160  4.387771e-01  5.090221e-01  4.551925e-01  4.964428e-01
161  1.742646e-02  1.456154e-02  1.570992e-02  1.524960e-02
162  1.374812e-01  3.847141e-02  1.141456e-01  5.630707e-02
163  1.846173e+00  1.846371e+00  1.846373e+00  1.846373e+00

[164 rows x 4 columns]


In [6]:
result = pd.concat([popetremb,predictions_df],axis=1)
result['PRS_REM_MNT_2025'] = result['ratio2025']*result['2025']
result['PRS_REM_MNT_2026'] = result['ratio2026']*result['2026']
result['PRS_REM_MNT_2027'] = result['ratio2027']*result['2027']
result['PRS_REM_MNT_2028'] = result['ratio2028']*result['2028']
result.to_csv('resultat.csv',sep=',')

In [ ]:
import pandas as pd

# Chargement initial (2017)
concat1724 = pd.read_csv('concat2017.csv', sep=';')
if 'Unnamed: 0' in concat1724.columns:
    concat1724 = concat1724.drop(columns=['Unnamed: 0'])

# Boucle de fusion pour les années suivantes
for i in range(2018, 2025):
    temp_df = pd.read_csv(f'concat{i}.csv', sep=';')
    
    # Supprimer la colonne Unnamed: 0 si elle est présente
    if 'Unnamed: 0' in temp_df.columns:
        temp_df = temp_df.drop(columns=['Unnamed: 0'])
        
    # Fusion avec le DataFrame principal
    concat1724 = concat1724.merge(
        temp_df,
        how='outer',
        on=['AGE_BEN_SNDS', 'ASU_NAT', 'CPT_ENV_TYP', 'PRS_NAT'],
        suffixes=(f'_{i-1}', f'_{i}')
    )


In [18]:
parprs = final_df[['PRS_REM_MNT','PRS_NAT']].groupby(['PRS_NAT']).sum().reset_index()

In [20]:
parprs.to_csv('parprs.csv')

In [14]:
pd.read_csv('concat24.csv',sep=';')

,Unnamed: 0,AGE_BEN_SNDS,ASU_NAT,CPT_ENV_TYP,PRS_NAT,PRS_REM_MNT
0,0,0,40,0,0,64231.68
1,1,0,40,0,1,5437.08
2,2,0,40,1,0,5818006.37
3,3,0,40,1,1,30651214.06
4,4,0,40,1,2,650.00
...,...,...,...,...,...,...
105,105,99,40,3,0,44367.00
106,106,99,40,3,1,832530.87
107,107,99,40,9,0,546442.47
108,108,99,40,9,1,1856488.45
